# Amazon Personalize：使用项目元数据识别更强的相似性

您使用 Amazon Personalize 提供的推荐的相关性取决于生成推荐时可用的数据。Amazon Personalize 使用用户的历史交互、项目的属性和用户的元数据来了解哪些项目与每个用户最相关。Amazon Personalize 所需的主要数据是用户-项目交互数据。用户与目录中项目的交互，例如单击产品、阅读文章、观看视频或购买产品，是他们过去找到相关项目的重要信号。添加项目和用户属性（也称为元数据），可以增强推荐的相关性；尤其是对于与用户找到的相关项目相似的新项目。但是，结构化元数据（例如项目的类别、样式或流派）可能并不总是现成可用，或者无法提供叙事描述中的所有信息。

现在，Amazon Personalize 允许您添加项目元数据，如产品描述、电影流派等。Amazon Personalize 可以托管、管理和自动处理您的项目元数据属性，并使用这些数据来提高 Amazon Personalize 相关项目解决方案的效果。

此笔记本将演示与 `SIMS` 配方相比，新配方 `aws-similar-items` 加项目元数据如何提高推荐的相关性。

来自 Amazon Prime Pantry 类别的 Amazon Reviews 数据用于交互和项目数据集。

考虑在项目数据集中包含文本时，请记住以下最佳做法。

将创建一个数据集组，其中包含交互数据和项目元数据。这样，我们就可以训练两个单独的模型，并比较其脱机和联机结果。

In [1]:
import pandas as pd
import json
import numpy as np
from datetime import datetime
import boto3
import time
from time import sleep
from lxml import html

## 加载和检查数据集

我们将首先加载 Prime Pantry 评论数据集。您需要填写表单才能访问数据文件：

http://deepyeti.ucsd.edu/jianmo/amazon/index.html

引文：
> 使用距离标记评论和细粒度方面来证明推荐的合理性  
> Jianmo Ni、Jiacheng Li、Julian McAuley  
> Empirical Methods in Natural Language Processing (EMNLP), 2019 [pdf](http://cseweb.ucsd.edu/~jmcauley/pdfs/emnlp19a.pdf)

In [2]:
data_dir = 'raw_data'
!mkdir $data_dir

!cd $data_dir && \
    wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFiles/Prime_Pantry.json.gz && \
    wget http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_Prime_Pantry.json.gz

--2021-10-07 15:14:39--  http://deepyeti.ucsd.edu/jianmo/amazon/categoryFiles/Prime_Pantry.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 45435146 (43M) [application/octet-stream]
Saving to: ‘Prime_Pantry.json.gz’

Prime_Pantry.json.g 100%[===================>]  43.33M  17.6MB/s    in 2.5s    

2021-10-07 15:14:42 (17.6 MB/s) - ‘Prime_Pantry.json.gz’ saved [45435146/45435146]

--2021-10-07 15:14:42--  http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_Prime_Pantry.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5281662 (5.0M) [application/octet-stream]
Saving to: ‘meta_Prime_Pantry.json.gz’

meta_Prime_Pantry.j 100%[===================>]   5.04M  7.24MB/s    

### 加载并检查评论数据

我们将首先加载 Prime Pantry 产品的评论数据集，并运行一些命令来查看需要处理的内容。

In [3]:
pantry_df = pd.read_json(data_dir + '/Prime_Pantry.json.gz', lines=True, compression='infer')
pantry_df.head()

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,image,style
0,5,True,"12 14, 2014",A1NKJW0TNRVS7O,B0000DIWNZ,Tamara M.,Good clinging,Clings well,1418515200,NaN,NaN,NaN
1,4,True,"11 20, 2014",A2L6X37E8TFTCC,B0000DIWNZ,Amazon Customer,Fantastic buy and a good plastic wrap. Even t...,Saran could use more Plus to Cling better.,1416441600,NaN,NaN,NaN
2,4,True,"10 11, 2014",A2WPR4W6V48121,B0000DIWNZ,noname,ok,Four Stars,1412985600,NaN,NaN,NaN
3,3,False,"09 1, 2014",A27EE7X7L29UMU,B0000DIWNZ,ZapNZs,Saran Cling Plus is kind of like most of the C...,"The wrap is fantastic, but the dispensing, cut...",1409529600,4,NaN,NaN
4,4,True,"08 10, 2014",A1OWT4YZGB5GV9,B0000DIWNZ,Amy Rogers,This is my go to plastic wrap so there isn't m...,has been doing it's job for years,1407628800,NaN,NaN,NaN


In [4]:
pantry_df.shape

(471614, 12)

我们可以从此输出中学到什么？ 有超过 47.1 万条评论和 12 列数据。`asin` 列是我们的唯一项目标识符，`reviewerID` 是我们的唯一用户标识符，`unixReviewTime` 是我们的评论时间戳，`overall` 表示按照 1-5 的等级对评论积极性的评级。我们将使用此文件作为 Personalize 交互数据集的基础。 

### 构建和保存交互数据集

让我们通过减少我们想要包含的行来开始构建我们的交互数据集。第一步是只分离积极的评论。为此，我们将假设总体评级为 4 或更高的任何评论都是积极的评论。评级为 3 或以下的任何评论都是一般或负面评论。

In [5]:
positive_reviews_df = pantry_df[pantry_df['overall'] > 3]
positive_reviews_df.shape

(387692, 12)

我们的正面评论已减少到 38.7 万条。仍然有足够的数据来训练 Personalize 模型。

接下来，让我们将数据集的范围缩小到所需的列，并添加一个 `EVENT_TYPE` 列来指示要捕获的事件类型。如果您选择这样做，现在添加一个 `EVENT_TYPE` 列将使以后探索测试实时事件更简单（因为 `eventType` 是 [PutEvents](https://docs.aws.amazon.com/personalize/latest/dg/API_UBS_PutEvents.html) API 的必填字段）。

In [6]:
positive_reviews_df = positive_reviews_df[['reviewerID', 'asin', 'unixReviewTime', 'overall']]
positive_reviews_df['EVENT_TYPE']='reviewed'

positive_reviews_df.head()

,reviewerID,asin,unixReviewTime,overall,EVENT_TYPE
0,A1NKJW0TNRVS7O,B0000DIWNZ,1418515200,5,reviewed
1,A2L6X37E8TFTCC,B0000DIWNZ,1416441600,4,reviewed
2,A2WPR4W6V48121,B0000DIWNZ,1412985600,4,reviewed
4,A1OWT4YZGB5GV9,B0000DIWNZ,1407628800,4,reviewed
5,A1GN2ADKF1IE7K,B0000DIWNZ,1405296000,5,reviewed


我们应该做的最后一项检查是对 `unixReviewTime` 列值进行健全性检查。由于 Personalize 基于每个交互的日期和时间构建序列模型，因此，以预期的格式表示每个交互的时间戳非常重要，以便能够正确解释。

让我们为 `unixReviewTime` 列选择一个值并将该值解析为人类可读的日期，以便我们可以验证它是否合理。

In [7]:
time_stamp = positive_reviews_df.iloc[50]['unixReviewTime']
print(time_stamp)
print(datetime.utcfromtimestamp(time_stamp).strftime('%Y-%m-%d %H:%M:%S'))

1321488000
2011-11-17 00:00:00


时间戳值看起来很好。让我们获取数据集的一些最终摘要信息。

In [8]:
positive_reviews_df.describe(include='all')

,reviewerID,asin,unixReviewTime,overall,EVENT_TYPE
count,387692,387692,3.876920e+05,387692.000000,387692
unique,202254,10584,NaN,NaN,1
top,A35Q0RBM3YNQNF,B00XA9DADC,NaN,NaN,reviewed
freq,176,5288,NaN,NaN,387692
mean,NaN,NaN,1.468847e+09,4.847227,NaN
std,NaN,NaN,4.314975e+07,0.359769,NaN
min,NaN,NaN,1.073693e+09,4.000000,NaN
25%,NaN,NaN,1.447200e+09,5.000000,NaN
50%,NaN,NaN,1.474718e+09,5.000000,NaN
75%,NaN,NaN,1.498435e+09,5.000000,NaN


我们有 20.2 万不同评论者/用户针对 1 万个不同产品的 38.7 条评论。这是我们交互数据集的基础。

但是，在我们将这些信息用作交互数据集之前，我们需要重命名列以匹配 Personalize 所要求的列。

In [9]:
positive_reviews_df.rename(columns = {'reviewerID':'USER_ID', 'asin':'ITEM_ID', 
                              'unixReviewTime':'TIMESTAMP', 'overall': 'EVENT_VALUE'}, inplace = True)
positive_reviews_df

,USER_ID,ITEM_ID,TIMESTAMP,EVENT_VALUE,EVENT_TYPE
0,A1NKJW0TNRVS7O,B0000DIWNZ,1418515200,5,reviewed
1,A2L6X37E8TFTCC,B0000DIWNZ,1416441600,4,reviewed
2,A2WPR4W6V48121,B0000DIWNZ,1412985600,4,reviewed
4,A1OWT4YZGB5GV9,B0000DIWNZ,1407628800,4,reviewed
5,A1GN2ADKF1IE7K,B0000DIWNZ,1405296000,5,reviewed
...,...,...,...,...,...
471609,A19GSVHXVT5NNF,B01HI8JVI8,1494892800,5,reviewed
471610,ABSCTKLX9F9IU,B01HI8JVI8,1493769600,5,reviewed
471611,A2R33RCWKDHZ3L,B01HI8JVI8,1492646400,5,reviewed
471612,A2INGHYEXZDHMC,B01HI8JVI8,1492560000,5,reviewed


最后，让我们将正面评论数据框另存为 CSV。我们稍后会在此笔记本中将此 CSV 上载到 Personalize。

In [10]:
interactions_filename = "interactions.csv"
positive_reviews_df.to_csv(interactions_filename, index=False, float_format='%.0f')

### 加载和检查项目元数据

现在，我们已经建立了交互数据集，让我们看一下项目数据集。在这里，我们将找到我们将包含在模型中的非结构化文本值。

与评论数据集一样，Prime Pantry 项目元数据文件也以 JSON 表示。由于此文件的嵌套性质，这将在按照我们需要的方式格式化数据方面带来一些挑战。

让我们首先将元数据文件加载到数据框中并查看数据。

In [11]:
pantry_meta_df = pd.read_json('raw_data/meta_Prime_Pantry.json.gz', lines=True, compression='infer')
pantry_meta_df

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes
0,[],,[Sink your sweet tooth into MILK DUDS Candya d...,,"HERSHEY'S Milk Duds Candy, 5 Ounce(Halloween C...","[B019KE37WO, B007NQSWEU]",,Milk Duds,[],[],[],"{'ASIN: ': 'B00005BPJO', 'Item model number:':...","<img src=""https://m.media-amazon.com/images/G/...",,NaT,$5.00,B00005BPJO,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
1,[],,[Sink your sweet tooth into MILK DUDS Candya d...,,"HERSHEY'S Milk Duds Candy, 5 Ounce(Halloween C...","[B019KE37WO, B007NQSWEU]",,Milk Duds,[],[],[],"{'ASIN: ': 'B00005BPJO', 'Item model number:':...","<img src=""https://m.media-amazon.com/images/G/...",,NaT,$5.00,B00005BPJO,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
2,[],,[A perfect Lentil soup starts with Goya Lentil...,,"Goya Dry Lentils, 16 oz","[B003SI144W, B000VDRKEK]",,Goya,[],[],"[B074MFVZG7, B079PTH69L, B000VDRKEK, B074M9T81...",{'ASIN: ': 'B0000DIF38'},"<img src=""https://images-na.ssl-images-amazon....",,NaT,,B0000DIF38,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
3,[],,[Saran Premium Wrap is an extra tough yet easy...,,"Saran Premium Plastic Wrap, 100 Sq Ft","[B01MY5FHT6, B000PYF8VM, B000SRMDFA, B07CX6LN8...",,Saran,[],[],"[B077QLSLRQ, B00JPKW1RQ, B000FE2IK6, B00XUJHJ9...",{'Domestic Shipping: ': 'This item can only be...,"<img src=""https://images-na.ssl-images-amazon....",,NaT,,B0000DIWNI,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
4,[],,[200 sq ft (285 ft x 11-3/4 in x 18.6 m2). Eas...,,"Saran Cling Plus Plastic Wrap, 200 Sq Ft",[],,Saran,[],[],[B0014CZ0TE],{'Domestic Shipping: ': 'This item can only be...,"<img src=""https://images-na.ssl-images-amazon....",,NaT,,B0000DIWNZ,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10808,[],,[These bars are where our journey started and ...,,"KIND Bars, Caramel Almond &amp; Sea Salt, Glut...",[],,KIND,[],"26,259 in Grocery & Gourmet Food (","[B00JQQAN60, B00JQQAWSY, B0111K7V54, B0111K8L9...","{'ASIN: ': 'B01HI76312', 'Item model number:':...","<img src=""https://images-na.ssl-images-amazon....",,NaT,$3.98,B01HI76312,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
10809,[],,[These bars are where our journey started and ...,,"KIND Bars, Maple Glazed Pecan &amp; Sea Salt, ...",[],,KIND,[],"16,822 in Grocery & Gourmet Food (","[B0111K97JC, B00JQQAN60, B0111K8L9Y, B01HI7631...",{'ASIN: ': 'B01HI76790'},"<img src=""https://images-na.ssl-images-amazon....",,NaT,$5.81,B01HI76790,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
10810,[],,[These bars are where our journey started and ...,,"KIND Bars, Dark Chocolate Almond &amp; Coconut...",[],,KIND,[],"107,057 in Grocery & Gourmet Food (","[B0111K7V54, B01HI76312, B00JQQAL0S, B0111K97J...",{'ASIN: ': 'B01HI76SA8'},"<img src=""https://images-na.ssl-images-amazon....",,NaT,$4.98,B01HI76SA8,[],[]
10811,[],,[These bars are where our journey started and ...,,"KIND Bars, Honey Roasted Nuts &amp; Sea Salt, ...",[],,KIND,[],"24,648 in Grocery & Gourmet Food (","[B00JQQAN60, B0111K7V54, B01HI76312, B0111K97J...",{'ASIN: ': 'B01HI76XS0'},"<img src=""https://images-na.ssl-images-amazon....",,NaT,$5.81,B01HI76XS0,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...


In [12]:
pantry_meta_df.describe()

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes
count,10813,10813,10813,10813,10813,10813,10813,10813,10813,10813,10813,10813,10813,10813,0,10813,10813,10813,10813
unique,1,1,9409,1,10782,3957,1,1960,763,4828,5940,10786,4,1,0,1482,10812,8940,8940
top,[],,[],,"Centrum Adult Flavor Burst (120 Count, Mixed F...",[],,L'Oreal Paris,[],[],[],{},"<img src=""https://images-na.ssl-images-amazon....",,NaN,,B00005BPJO,[],[]
freq,10813,10813,98,10813,2,6754,10813,171,9777,5937,4835,24,10621,10813,NaN,4063,2,1781,1781


那么，我们可以从这些信息中了解到什么？ 首先，元数据文件中包含了超过 1 万种产品。大多数列对我们使用 Personalize 没有什么价值，因为它们与功能（图像 URL、`details`、`also_viewed`、`also_buy` 等）无关，或者大部分是空白/稀疏的列（`category`、`fit`、`tech1` 等）。`asin` 列是每个项目的唯一标识符（尽管看起来有一个重复），`brand` 和 `price` 看起来可能很有用。`description` 列是我们将用于非结构化文本的内容。

然而，我们必须对要在项目数据集中使用的字段进行一些清理和重新格式化。例如，由于这些值的表示方式和从原始 JSON 文件中解析的方式，`price` 字段是格式化的货币值（字符串）而不是数字，`description` 字段被加载为字符串数组。最后，`description` 值还包含需要剥离的 HTML 标记。

让我们首先创建一个数据框，其中仅包含项目数据集所需的列。

In [13]:
items_df = pantry_meta_df.copy()
items_df = items_df[['asin', 'brand', 'price', 'description']]
items_df.head(10)

,asin,brand,price,description
0,B00005BPJO,Milk Duds,$5.00,[Sink your sweet tooth into MILK DUDS Candya d...
1,B00005BPJO,Milk Duds,$5.00,[Sink your sweet tooth into MILK DUDS Candya d...
2,B0000DIF38,Goya,,[A perfect Lentil soup starts with Goya Lentil...
3,B0000DIWNI,Saran,,[Saran Premium Wrap is an extra tough yet easy...
4,B0000DIWNZ,Saran,,[200 sq ft (285 ft x 11-3/4 in x 18.6 m2). Eas...
5,B0000GH6UG,Ibarra,,"[Ibarra Chocolate, 19 Oz, , ]"
6,B0000KC2BK,Knorr,$3.09,[Knorr Granulated Chicken Flavor Bouillon is a...
7,B0001E1IN8,Castillo,,[Red chili habanero sauces. They are present t...
8,B00032E8XK,Chicken of the Sea,$1.48,[Chicken of the Sea Solid White Albacore Tuna ...
9,B0005XMTHE,Smucker's,$2.29,"[Helps build muscles with bcaa's amino acids, ..."


接下来，让我们根据 `asin` 列值删除重复的行。根据上面的 `describe()` 输出，应该只有一个副本。

In [14]:
items_df = items_df.drop_duplicates(subset=['asin'], keep='last')
items_df.shape

(10812, 4)

接下来，让我们专注于重新格式化和清理 `description` 列值。正如您在上面看到的，`description` 当前表示为字符串数组（因为这就是它在 JSON 文件中的表示方式）。我们需要将该数组扁平化为单个字符串，并从每个片段中剥离所有 HTML 标记。

我们将首先创建两个实用工具函数用于清理 `description`（稍后创建我们想要显示推荐产品的标题时原始数据集中的 `title` 列）。

In [15]:
# Strips and cleans a value of HTML markup and whitespace.
def clean_markup(value):
    s = str(value).strip()
    if s != '':
        s = str(html.fromstring(s).text_content())
        s = ' '.join(s.split())
                
    return s.strip()

# Cleans and reformats the description column value for a dataframe row.
def clean_and_reformat_description(row):
    s = ''
    for el in row['description']:
        el = clean_markup(el)
        if el != '':
            s += ' ' + el
                
    return s.strip()

In [16]:
items_df['description'] = items_df.apply(clean_and_reformat_description, axis=1)
items_df

,asin,brand,price,description
1,B00005BPJO,Milk Duds,$5.00,Sink your sweet tooth into MILK DUDS Candya de...
2,B0000DIF38,Goya,,A perfect Lentil soup starts with Goya Lentils...
3,B0000DIWNI,Saran,,Saran Premium Wrap is an extra tough yet easy ...
4,B0000DIWNZ,Saran,,200 sq ft (285 ft x 11-3/4 in x 18.6 m2). Easy...
5,B0000GH6UG,Ibarra,,"Ibarra Chocolate, 19 Oz"
...,...,...,...,...
10808,B01HI76312,KIND,$3.98,These bars are where our journey started and i...
10809,B01HI76790,KIND,$5.81,These bars are where our journey started and i...
10810,B01HI76SA8,KIND,$4.98,These bars are where our journey started and i...
10811,B01HI76XS0,KIND,$5.81,These bars are where our journey started and i...


接下来，让我们看一下 `price` 列并将该列类型从字符串更改为浮点数。

In [17]:
items_df['price'].value_counts()

          4063
$2.99      114
$3.99      113
$4.99      103
$5.99       87
          ... 
$10.40       1
$43.53       1
$20.42       1
$17.74       1
$62.99       1
Name: price, Length: 1482, dtype: int64

以下单元格会将空/非数字价格转换为 `np.nan`，所有其他单元格将删除 `$` 货币符号。这将允许我们将类型强制为浮点数。

In [18]:
def convert_price(row):
    v = str(row['price']).strip().replace('$', '')
    if v == '' or not v.lstrip('-').replace('.', '').isdigit():
        return np.nan
    return v

items_df['price'] = items_df.apply(convert_price, axis=1)
items_df

,asin,brand,price,description
1,B00005BPJO,Milk Duds,5.00,Sink your sweet tooth into MILK DUDS Candya de...
2,B0000DIF38,Goya,NaN,A perfect Lentil soup starts with Goya Lentils...
3,B0000DIWNI,Saran,NaN,Saran Premium Wrap is an extra tough yet easy ...
4,B0000DIWNZ,Saran,NaN,200 sq ft (285 ft x 11-3/4 in x 18.6 m2). Easy...
5,B0000GH6UG,Ibarra,NaN,"Ibarra Chocolate, 19 Oz"
...,...,...,...,...
10808,B01HI76312,KIND,3.98,These bars are where our journey started and i...
10809,B01HI76790,KIND,5.81,These bars are where our journey started and i...
10810,B01HI76SA8,KIND,4.98,These bars are where our journey started and i...
10811,B01HI76XS0,KIND,5.81,These bars are where our journey started and i...


In [19]:
items_df['price'].value_counts()

2.99     114
3.99     113
4.99     103
5.99      87
2.98      76
        ... 
12.66      1
24.82      1
19.92      1
13.97      1
13.33      1
Name: price, Length: 1480, dtype: int64

In [20]:
items_df['price'] = items_df['price'].astype(float)

接下来，我们将重命名列以匹配 Personalize 要求的名称和大写名称格式。

In [21]:
items_df.rename(columns = {'asin':'ITEM_ID', 'brand':'BRAND', 
                              'price':'PRICE', 'description': 'DESCRIPTION'}, inplace = True)
items_df.head(10)

,ITEM_ID,BRAND,PRICE,DESCRIPTION
1,B00005BPJO,Milk Duds,5.00,Sink your sweet tooth into MILK DUDS Candya de...
2,B0000DIF38,Goya,NaN,A perfect Lentil soup starts with Goya Lentils...
3,B0000DIWNI,Saran,NaN,Saran Premium Wrap is an extra tough yet easy ...
4,B0000DIWNZ,Saran,NaN,200 sq ft (285 ft x 11-3/4 in x 18.6 m2). Easy...
5,B0000GH6UG,Ibarra,NaN,"Ibarra Chocolate, 19 Oz"
6,B0000KC2BK,Knorr,3.09,Knorr Granulated Chicken Flavor Bouillon is a ...
7,B0001E1IN8,Castillo,NaN,Red chili habanero sauces. They are present to...
8,B00032E8XK,Chicken of the Sea,1.48,Chicken of the Sea Solid White Albacore Tuna i...
9,B0005XMTHE,Smucker's,2.29,"Helps build muscles with bcaa's amino acids, i..."
10,B0005XNE6E,Snapple,1.99,"At Snapple, we believe lifes a peach. Weve bee..."


我们将创建一个项目 CSV。我们将使用此文件来训练我们的个性化模型，以便我们可以比较离线指标，并对推荐进行一些在线检查。

In [22]:
items_filename = "items-metadata.csv"
items_df.to_csv(items_filename, index=False, float_format='%.2f')

## 创建数据集组并上载数据集

有了我们需要构建的数据集，现在该使用数据集导入作业将它们上载到 Personalize 了。上载 CSV 之前，我们需要创建数据集组，为数据集创建架构，并创建数据集。

我们将首先创建需要与 Personalize 交互的 SDK 客户端。

### 创建数据集组

让我们创建数据集组。

In [24]:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

In [25]:
create_dataset_group_response = personalize.create_dataset_group(
    name = "amazon-pantry-aws-similar-items"
)

dataset_group = create_dataset_group_response['datasetGroupArn']
print(json.dumps(create_dataset_group_response, indent=2))

{
  "datasetGroupArn": "arn:aws:personalize:us-east-1:144386903708:dataset-group/amazon-pantry-aws-similar-items",
  "ResponseMetadata": {
    "RequestId": "4b22f3c9-d00d-49fa-b56f-3049e60b14e2",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Thu, 07 Oct 2021 15:17:26 GMT",
      "x-amzn-requestid": "4b22f3c9-d00d-49fa-b56f-3049e60b14e2",
      "content-length": "110",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


由于数据集组可能需要几秒钟才能完全创建，所以，让我们等到数据集组都处于 ACTIVE 状态。

In [26]:
in_progress_dataset_group_arns = [ dataset_group ]

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    for dataset_group_arn in in_progress_dataset_group_arns:
        describe_dataset_group_response = personalize.describe_dataset_group(
            datasetGroupArn = dataset_group_arn
        )
        status = describe_dataset_group_response["datasetGroup"]["status"]
        if status == "ACTIVE":
            print("Dataset group create succeeded for {}".format(dataset_group_arn))
            in_progress_dataset_group_arns.remove(dataset_group_arn)
        elif status == "CREATE FAILED":
            print("Create failed for {}".format(dataset_group_arn))
            in_progress_dataset_group_arns.remove(dataset_group_arn)

    if len(in_progress_dataset_group_arns) <= 0:
        break
    else:
        print("At least one dataset group create is still in progress")
                
    time.sleep(10)

At least one dataset group create is still in progress
At least one dataset group create is still in progress
Dataset group create succeeded for arn:aws:personalize:us-east-1:144386903708:dataset-group/amazon-pantry-aws-similar-items


### 创建交互数据集架构和数据集

我们将为交互数据集类型创建一个架构，并在两个解决方案版本之间共享该架构。这是可能的，因为架构对于 AWS 账户是全局的，而不是特定于数据集组。

In [27]:
interactions_schema = schema = {
    "type": "record",
    "name": "Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "TIMESTAMP",
            "type": "long"
        },
        {
            "name": "EVENT_VALUE",
            "type": "float"
        },
        {
            "name": "EVENT_TYPE",
            "type": "string"
        }
    ],
    "version": "1.0"
}
            
create_schema_response = personalize.create_schema(
    name = "amazon-pantry-interactions-1",
    schema = json.dumps(interactions_schema)
)

interaction_schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

{
  "schemaArn": "arn:aws:personalize:us-east-1:144386903708:schema/amazon-pantry-interactions-1",
  "ResponseMetadata": {
    "RequestId": "96501057-36b8-4ca7-ad76-911498441b14",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Thu, 07 Oct 2021 15:18:31 GMT",
      "x-amzn-requestid": "96501057-36b8-4ca7-ad76-911498441b14",
      "content-length": "94",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


接下来，我们将在数据集组中创建一个交互数据集，指定刚才创建的架构。

In [28]:
dataset_type = "INTERACTIONS"
create_dataset_response = personalize.create_dataset(
    name = "amazon-pantry-ints",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group,
    schemaArn = interaction_schema_arn
)

interactions_dataset = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

{
  "datasetArn": "arn:aws:personalize:us-east-1:144386903708:dataset/amazon-pantry-aws-similar-items/INTERACTIONS",
  "ResponseMetadata": {
    "RequestId": "f3aa8827-d8a3-4afa-b731-70ccd4595675",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Thu, 07 Oct 2021 15:18:36 GMT",
      "x-amzn-requestid": "f3aa8827-d8a3-4afa-b731-70ccd4595675",
      "content-length": "112",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### S3 中的阶段交互 CSV

在将之前创建的交互 CSV 上载到刚刚创建的 Personalize 数据集中之前，我们需要将此 CSV 暂存到 S3 存储桶中。

让我们创建一个 S3 存储桶并将交互 CSV 文件复制到存储桶中。

In [29]:
# Determine the current S3 region where this notebook is being hosted in SageMaker.
with open('/opt/ml/metadata/resource-metadata.json') as notebook_info:
    data = json.load(notebook_info)
    resource_arn = data['ResourceArn']
    region = resource_arn.split(':')[3]
print(region)

us-east-1


In [30]:

s3 = boto3.client('s3')
account_id = boto3.client('sts').get_caller_identity().get('Account')
bucket_name = "amazon-pantry-personalize-example"+account_id
print(bucket_name)
if region == "us-east-1":
    s3.create_bucket(Bucket=bucket_name)
else:
    region='us-west-2'
    s3.create_bucket(
        Bucket=bucket_name,
        CreateBucketConfiguration={'LocationConstraint': region}
    )

amazon-pantry-personalize-example144386903708


#### 将交互 CSV 上载到 S3

In [31]:
boto3.Session().resource('s3').Bucket(bucket_name).Object(interactions_filename).upload_file(interactions_filename)

### 创建 S3 存储桶策略和 IAM 角色

在我们向 Personalize 提交数据集导入作业之前，我们必须创建一个存储桶策略和 IAM 角色，以便 Personalize 可以访问我们的存储桶。

In [32]:
policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:*Object",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket_name),
                "arn:aws:s3:::{}/*".format(bucket_name)
            ]
        }
    ]
}

s3.put_bucket_policy(Bucket=bucket_name, Policy=json.dumps(policy))

{'ResponseMetadata': {'RequestId': 'H5K1X3F9BCJMVW04',
  'HostId': 'E+jhicBArPArPsseiSMoJyxiiUrhmypt9D5GVSCMY6EgYx1wFDYq4eYClVYInY1E6T5eiZO49EI=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': 'E+jhicBArPArPsseiSMoJyxiiUrhmypt9D5GVSCMY6EgYx1wFDYq4eYClVYInY1E6T5eiZO49EI=',
   'x-amz-request-id': 'H5K1X3F9BCJMVW04',
   'date': 'Thu, 07 Oct 2021 15:20:29 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

In [33]:
iam = boto3.client("iam")

role_name = "PersonalizeRoleAmazonPantryAwsSimilarItems"
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}

create_role_response = iam.create_role(
    RoleName = role_name,
    AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
)

# AmazonPersonalizeFullAccess provides access to any S3 bucket with a name that includes "personalize" or "Personalize" 
# if you would like to use a bucket with a different name, please consider creating and attaching a new policy
# that provides read access to your bucket or attaching the AmazonS3ReadOnlyAccess policy to the role
policy_arn = "arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess"
iam.attach_role_policy(
    RoleName = role_name,
    PolicyArn = policy_arn
)

# Now add S3 support
iam.attach_role_policy(
    PolicyArn='arn:aws:iam::aws:policy/AmazonS3FullAccess',
    RoleName=role_name
)
time.sleep(20) # wait for a minute to allow IAM role policy attachment to propagate

role_arn = create_role_response["Role"]["Arn"]
print(role_arn)

arn:aws:iam::144386903708:role/PersonalizeRoleAmazonPantryAwsSimilarItems


### 导入交互数据集

现在，我们已经准备好将 S3 存储桶中的阶段性交互 CSV 导入我们创建的 Personalize 数据集中。

In [35]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "amazon-pantry-interactions-import",
    datasetArn = interactions_dataset,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, interactions_filename)
    },
    roleArn = role_arn
)

dataset_import_job = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

{
  "datasetImportJobArn": "arn:aws:personalize:us-east-1:144386903708:dataset-import-job/amazon-pantry-interactions-import",
  "ResponseMetadata": {
    "RequestId": "b8894f08-58a4-428e-b1f6-5df4e59849b9",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Thu, 07 Oct 2021 15:22:04 GMT",
      "x-amzn-requestid": "b8894f08-58a4-428e-b1f6-5df4e59849b9",
      "content-length": "121",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### 创建项目数据集架构和数据集

接下来，我们将对项目数据集重复此过程。

创建一个包含此描述的架构。请务必记下 `DESCRIPTION` 字段上的 `"textual": True` 属性。这是您区分非结构化文本字段与分类字段和字符串字段的方式。如果没有此属性，Personalize 将不会应用自然语言处理技术从该文本中提取特征。

In [37]:
item_schema = {
    "type": "record",
    "name": "Items",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "BRAND",
            "type": [ "null", "string" ],
            "categorical": True
        },{
            "name": "PRICE",
            "type": [ "null", "float" ],
        },{
            "name": "DESCRIPTION",
            "type": [ "null", "string" ],
            "textual": True
        }
    ],
    "version": "1.0"
}

create_schema_response = personalize.create_schema(
    name = "amazon-pantry-items-schema",
    schema = json.dumps(item_schema)
)

item_schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

{
  "schemaArn": "arn:aws:personalize:us-east-1:144386903708:schema/amazon-pantry-items-schema",
  "ResponseMetadata": {
    "RequestId": "2bf13abf-6115-415d-8f7d-bf1c4482bfb4",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Thu, 07 Oct 2021 15:22:21 GMT",
      "x-amzn-requestid": "2bf13abf-6115-415d-8f7d-bf1c4482bfb4",
      "content-length": "92",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


接下来，我们将在数据集组中创建 Personalize 数据集，请特别注意为每个数据集指定相应的架构 ARN。

In [38]:
dataset_type = "ITEMS"
create_dataset_response = personalize.create_dataset(
    name = "amazon-pantry-items",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = item_schema_arn
)

items_dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

{
  "datasetArn": "arn:aws:personalize:us-east-1:144386903708:dataset/amazon-pantry-aws-similar-items/ITEMS",
  "ResponseMetadata": {
    "RequestId": "f304bdcb-d941-444b-a83e-857c35eb0f2e",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Thu, 07 Oct 2021 15:22:29 GMT",
      "x-amzn-requestid": "f304bdcb-d941-444b-a83e-857c35eb0f2e",
      "content-length": "105",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


#### S3 中的阶段项目 CSV

接下来，我们将两个项目 CSV 文件复制到上面创建的同一个 S3 存储桶中。

In [39]:
boto3.Session().resource('s3').Bucket(bucket_name).Object(items_filename).upload_file(items_filename)

### 为每个数据集组导入项目数据集

由于已设置 S3 存储桶策略和 IAM 角色，我们只需提交两个数据集导入作业即可导入项目 CSV。

In [40]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "amazon-pantry-items-import-job",
    datasetArn = items_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, items_filename)
    },
    roleArn = role_arn
)

dataset_import_job_items_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

{
  "datasetImportJobArn": "arn:aws:personalize:us-east-1:144386903708:dataset-import-job/amazon-pantry-items-import-job",
  "ResponseMetadata": {
    "RequestId": "27cf7ae3-1949-48fe-8884-bf282729cfc4",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Thu, 07 Oct 2021 15:22:45 GMT",
      "x-amzn-requestid": "27cf7ae3-1949-48fe-8884-bf282729cfc4",
      "content-length": "118",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### 等待交互数据集导入作业完成

以下单元格将等待两个导入作业完成。

In [ ]:
%%time

in_progress_import_arns = [ dataset_import_job ]

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    for import_arn in in_progress_import_arns:
        describe_dataset_import_job_response = personalize.describe_dataset_import_job(
            datasetImportJobArn = import_arn
        )
        status = describe_dataset_import_job_response["datasetImportJob"]['status']
        if status == "ACTIVE":
            print("Dataset import succeeded for {}".format(import_arn))
            in_progress_import_arns.remove(import_arn)
        elif status == "CREATE FAILED":
            print("Create failed for {}".format(import_arn))
            in_progress_import_arns.remove(import_arn)

    if len(in_progress_import_arns) <= 0:
        break
    else:
        print("At least one dataset import job is still in progress")
                
    time.sleep(60)

### 等待项目导入作业完成

以下逻辑将等到两个项目数据集完全导入每个数据集组。

In [ ]:
%%time

in_progress_import_arns = [ dataset_import_job_items_arn ]

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    for import_arn in in_progress_import_arns:
        describe_dataset_import_job_response = personalize.describe_dataset_import_job(
            datasetImportJobArn = import_arn
        )
        status = describe_dataset_import_job_response["datasetImportJob"]['status']
        if status == "ACTIVE":
            print("Dataset import succeeded for {}".format(import_arn))
            in_progress_import_arns.remove(import_arn)
        elif status == "CREATE FAILED":
            print("Create failed for {}".format(import_arn))
            in_progress_import_arns.remove(import_arn)

    if len(in_progress_import_arns) <= 0:
        break
    else:
        print("At least one dataset import job is still in progress")
                
    time.sleep(60)

## 创建解决方案和解决方案版本

将交互和项目数据集导入每个数据集组后，我们接下来将使用每个数据集组中的数据的 user-personalization 配方创建解决方案和解决方案版本。

首先，让我们列出可用的 Personalize 配方。

In [41]:
personalize.list_recipes()

{'recipes': [{'name': 'aws-hrnn',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-hrnn',
   'status': 'ACTIVE',
   'creationDateTime': datetime.datetime(2019, 6, 10, 0, 0, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(2021, 10, 2, 13, 24, 21, 632000, tzinfo=tzlocal())},
  {'name': 'aws-hrnn-coldstart',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-hrnn-coldstart',
   'status': 'ACTIVE',
   'creationDateTime': datetime.datetime(2019, 6, 10, 0, 0, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(2021, 10, 2, 13, 24, 21, 632000, tzinfo=tzlocal())},
  {'name': 'aws-hrnn-metadata',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-hrnn-metadata',
   'status': 'ACTIVE',
   'creationDateTime': datetime.datetime(2019, 6, 10, 0, 0, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(2021, 10, 2, 13, 24, 21, 632000, tzinfo=tzlocal())},
  {'name': 'aws-personalized-ranking',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-personalized-ranking',
  

在此笔记本中，我们将使用 `aws-sims` 和 `aws-similar-items` 配方来训练两种解决方案。

In [42]:
sims_recipe_arn = "arn:aws:personalize:::recipe/aws-sims"
similar_items_recipe_arn = "arn:aws:personalize:::recipe/aws-similar-items"

首先，我们将为每个配方创建解决方案和解决方案版本。

In [43]:
sims_create_solution_response = personalize.create_solution(
    name = "amazon-pantry-sims-solution-example",
    datasetGroupArn = dataset_group_arn,
    recipeArn = sims_recipe_arn
)

sims_solution_arn = sims_create_solution_response['solutionArn']

In [44]:
print(sims_solution_arn)

arn:aws:personalize:us-east-1:144386903708:solution/amazon-pantry-sims-solution-example


In [45]:
sims_solution_version_response = personalize.create_solution_version(
    solutionArn = sims_solution_arn
)

In [46]:
sims_solution_version_arn = sims_solution_version_response['solutionVersionArn']
print(json.dumps(sims_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:144386903708:solution/amazon-pantry-sims-solution-example/0fed38cd",
  "ResponseMetadata": {
    "RequestId": "f3d70620-3170-4700-acaa-a84de14de287",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Thu, 07 Oct 2021 15:34:41 GMT",
      "x-amzn-requestid": "f3d70620-3170-4700-acaa-a84de14de287",
      "content-length": "121",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


接下来，我们将在包含项目描述的数据集组中创建解决方案和解决方案版本。

In [47]:
similar_items_create_solution_response = personalize.create_solution(
    name = "amazon-pantry-aws-similar-items-solution-example",
    datasetGroupArn = dataset_group_arn,
    recipeArn = similar_items_recipe_arn
)

similar_items_solution_arn = similar_items_create_solution_response['solutionArn']

In [48]:
similar_items_solution_version_response = personalize.create_solution_version(
    solutionArn = similar_items_solution_arn
)

In [49]:
similar_items_solution_version_arn = similar_items_solution_version_response['solutionVersionArn']
print(json.dumps(similar_items_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:144386903708:solution/amazon-pantry-aws-similar-items-solution-example/23419275",
  "ResponseMetadata": {
    "RequestId": "7b802cd0-0ee2-4166-a1e6-f78dfac0600d",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Thu, 07 Oct 2021 15:34:49 GMT",
      "x-amzn-requestid": "7b802cd0-0ee2-4166-a1e6-f78dfac0600d",
      "content-length": "134",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### 等待解决方案版本变为活跃状态

最后，我们将等待创建解决方案版本完成。在此步骤中，Personalize 基于数据集和所选配方训练机器学习模型。Personalize 还会将交互数据集拆分为训练部分和评估部分，这样它就可以使用保留的数据根据训练的模型评估推荐的质量。

您会注意到，与数据集组中不包含描述数据的解决方案版本相比，数据集中包含描述数据的解决方案版本需要更长的训练时间。

In [50]:
%%time

in_progress_solution_versions = [
    sims_solution_version_arn,
    similar_items_solution_version_arn
]

max_time = time.time() + 10*60*60 # 10 hours
while time.time() < max_time:
    for solution_version_arn in in_progress_solution_versions:
        version_response = personalize.describe_solution_version(
            solutionVersionArn = solution_version_arn
        )
        status = version_response["solutionVersion"]["status"]
        
        if status == "ACTIVE":
            print("Build succeeded for {}".format(solution_version_arn))
            in_progress_solution_versions.remove(solution_version_arn)
        elif status == "CREATE FAILED":
            print("Build failed for {}".format(solution_version_arn))
            in_progress_solution_versions.remove(solution_version_arn)
    
    if len(in_progress_solution_versions) <= 0:
        break
    else:
        print("At least one solution build is still in progress")
        
    time.sleep(60)

At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solutio

一般来说，添加基于文本的非结构化元数据会增加训练时间。在我们的案例中，您可以在上面看到，与基于不包含生产描述的数据集进行训练的解决方案版本相比，基于包含产品描述的数据集进行训练的解决方案版本所用的时间大约多 15 分钟。这种差异会因数据集的组成和文本值而异。

让我们检查每个解决方案版本的训练小时数并进行比较。

In [51]:
response = personalize.describe_solution_version(solutionVersionArn = sims_solution_version_arn)
training_hours_sims = response['solutionVersion']['trainingHours']

response = personalize.describe_solution_version(solutionVersionArn = similar_items_solution_version_arn)
training_hours_similar_items = response['solutionVersion']['trainingHours']
training_diff = (training_hours_sims - training_hours_similar_items) / training_hours_similar_items

print(f"Training hours sims: {training_hours_sims}")
print(f"Training hours similar items: {training_hours_similar_items}")

print("Difference of {:.2%}".format(training_diff))

Training hours sims: 1.607
Training hours similar items: 2.526
Difference of -36.38%


用于成本计算的训练小时数比用于具有描述列的数据集的训练小时数增加了大约 50%。

挂钟时间和训练小时数将根据数据集大小而有所不同，但此信息可以帮助您在考虑向数据集添加非结构化文本时评估权衡。

In [52]:
sims_solution = {
        "solution_arn": sims_solution_arn,
        "solution_version_arn": sims_solution_version_arn
}
sims_v2_solution = {
        "solution_arn": similar_items_solution_arn,
        "solution_version_arn": similar_items_solution_version_arn
}

In [53]:
def create_campaign(solution,name):
    create_campaign_response = personalize.create_campaign(
        name = "personalize-demo-" + name + "example",
        solutionVersionArn = solution['solution_version_arn'],
        minProvisionedTPS = 1
    )

    campaign_arn = create_campaign_response['campaignArn']
    print('campaign_arn:' + campaign_arn)
    return campaign_arn

def waitForCampaign(solution):
    max_time = time.time() + 3*60*60 # 3 hours
    while time.time() < max_time:
        describe_campaign_response = personalize.describe_campaign(
            campaignArn = solution['campaign_arn']
        )
        status = describe_campaign_response["campaign"]["status"]
        print("Campaign: {} {}".format(solution['campaign_arn'], status))

        if status == "ACTIVE" or status == "CREATE FAILED":
            break

        time.sleep(60)

#### 创建并等待 2 个活动
为每个项目相似性配方创建活动，但保持所有其他设置不变，以演示添加元数据的影响。

In [54]:
sims_solution['campaign_arn'] = create_campaign(sims_solution, 'sims')
sims_v2_solution['campaign_arn'] = create_campaign(sims_v2_solution, 'aws-similar-items')

campaign_arn:arn:aws:personalize:us-east-1:144386903708:campaign/personalize-demo-simsexample
campaign_arn:arn:aws:personalize:us-east-1:144386903708:campaign/personalize-demo-aws-similar-itemsexample


In [55]:
waitForCampaign(sims_solution)
waitForCampaign(sims_v2_solution)

Campaign: arn:aws:personalize:us-east-1:144386903708:campaign/personalize-demo-simsexample CREATE PENDING
Campaign: arn:aws:personalize:us-east-1:144386903708:campaign/personalize-demo-simsexample CREATE IN_PROGRESS
Campaign: arn:aws:personalize:us-east-1:144386903708:campaign/personalize-demo-simsexample CREATE IN_PROGRESS
Campaign: arn:aws:personalize:us-east-1:144386903708:campaign/personalize-demo-simsexample CREATE IN_PROGRESS
Campaign: arn:aws:personalize:us-east-1:144386903708:campaign/personalize-demo-simsexample CREATE IN_PROGRESS
Campaign: arn:aws:personalize:us-east-1:144386903708:campaign/personalize-demo-simsexample CREATE IN_PROGRESS
Campaign: arn:aws:personalize:us-east-1:144386903708:campaign/personalize-demo-simsexample CREATE IN_PROGRESS
Campaign: arn:aws:personalize:us-east-1:144386903708:campaign/personalize-demo-simsexample CREATE IN_PROGRESS
Campaign: arn:aws:personalize:us-east-1:144386903708:campaign/personalize-demo-simsexample CREATE IN_PROGRESS
Campaign: arn:

# 获得推荐

首先，我们将选择三种类型的项目进行推理
1. 具有大量交互的项目
1. 具有少量交互的项目
1. 随机项目 ID

然后，我们将了解对于提供的每个项目，每个模型的行为方式


In [57]:
items_interacted_df

,USER_ID,ITEM_ID,TIMESTAMP,EVENT_VALUE,EVENT_TYPE
0,A1NKJW0TNRVS7O,B0000DIWNZ,1418515200,5,reviewed
1,A2L6X37E8TFTCC,B0000DIWNZ,1416441600,4,reviewed
2,A2WPR4W6V48121,B0000DIWNZ,1412985600,4,reviewed
4,A1OWT4YZGB5GV9,B0000DIWNZ,1407628800,4,reviewed
5,A1GN2ADKF1IE7K,B0000DIWNZ,1405296000,5,reviewed
...,...,...,...,...,...
471609,A19GSVHXVT5NNF,B01HI8JVI8,1494892800,5,reviewed
471610,ABSCTKLX9F9IU,B01HI8JVI8,1493769600,5,reviewed
471611,A2R33RCWKDHZ3L,B01HI8JVI8,1492646400,5,reviewed
471612,A2INGHYEXZDHMC,B01HI8JVI8,1492560000,5,reviewed


让我们看一下我们的交互数据集并绘制与之交互的项目的分布情况

In [58]:
items_interacted_df = positive_reviews_df.copy()
# Getting unique aisns counts
asin_interaction_count = items_interacted_df['ITEM_ID'].value_counts()
# Transforming panda series to df
asin_interaction_count_df = pd.DataFrame({'asin':asin_interaction_count.index, 'count':asin_interaction_count.values})
asin_interaction_count_df

,asin,count
0,B00XA9DADC,5288
1,B00I3MVY9I,5086
2,B00W6T13OU,4023
3,B0014CZUZ8,3225
4,B0014CSIG6,2600
...,...,...
10579,B0160P77Q8,1
10580,B00ZKUD5D4,1
10581,B00IGB7ZM6,1
10582,B004WP217O,1


In [59]:
asin_interaction_count_df.describe(include='int')

,count
count,10584.000000
mean,36.630008
std,123.480233
min,1.000000
25%,4.000000
50%,12.000000
75%,33.250000
max,5288.000000


如上所述，交互次数最多的项目有约 5 千次交互，让我们绘制所有项目的分布图。在这里我们可以看到，有些项目的交互次数很高，有些项目的交互次数非常低甚至没有交互。

<AxesSubplot:>

In [60]:
asin_interaction_count_df.plot()

下面，我们可以仔细查看具有 100 到 300 次交互的项目。这些将在测试时为我们提供最可变的结果。

<AxesSubplot:>

In [61]:
zoom_interactions = asin_interaction_count_df.copy()
zoom_interactions = zoom_interactions.loc[(zoom_interactions["count"] > 100) & (zoom_interactions["count"] < 300)]
zoom_interactions.plot()

现在，让我们定义一些函数来探索项目元数据

In [70]:
# https://www.geeksforgeeks.org/how-to-select-rows-from-a-dataframe-based-on-column-values/
def get_item_brand(item_id):
    """
    Takes in an ID, returns its brand
    """

    return items_df.query('ITEM_ID=="{}"'.format(item_id))['BRAND'].item()

def get_item_price(item_id):
    """
    Takes in an ID, returns its brand
    """

    return items_df.query('ITEM_ID=="{}"'.format(item_id))['PRICE'].item()

def get_item_description(item_id):
    """
    Takes in an ID, returns its brand
    """

    return items_df.query('ITEM_ID=="{}"'.format(item_id))['DESCRIPTION'].item()
def get_item_df(item_id):
    """
    Takes in an ID, returns a title
    """
    temp_df = items_df.query('ITEM_ID=="{}"'.format(item_id))
  
    temp_df['INTERACTIONS_COUNT'] = get_item_count(item_id)
    return temp_df
def get_item_count(item_id):
    return asin_interaction_count_df.query('asin=="{}"'.format(item_id))['count'].item()
    
def get_recs_df(item_id, campaign):
    response = personalize_runtime.get_recommendations(
        campaignArn=campaign,
        itemId=item_id,
        numResults=10
    )
    return clean_recs_list(response['itemList'])

def clean_recs_list(rec_list):
    items = []
    for each in rec_list:
        items.append([each['itemId'], get_item_brand(each['itemId']), get_item_price(each['itemId']), get_item_description(each['itemId']), get_item_count(each['itemId'])])
    return pd.DataFrame (items, columns = ['ITEM_ID', 'BRAND', 'PRICE','DESCRIPTION', 'INTERACTIONS_COUNT'])

让我们获取一个随机项目 ID 并探索每个模型推荐

In [63]:
asin_interaction_count_df.sample()['asin'].item()

'B012X39R5K'

In [71]:
asin = asin_interaction_count_df.sample(1)['asin'].item()
recommended_item_df = get_item_df(asin)
recommendations_sims_df = get_recs_df(asin,sims_solution['campaign_arn'])
recommendations_sims_v2_df = get_recs_df(asin,sims_v2_solution['campaign_arn'])

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


这是我们将要对其进行推理的项目，正如我们在这里看到的那样，我们有品牌、价格和交互次数。理论上，对于交互次数较低的项目，SIMS 将返回最受欢迎的项目，而新的 aws-item-similarity 配方将返回与当前项目元数据更相关的项目

In [72]:
recommended_item_df

,ITEM_ID,BRAND,PRICE,DESCRIPTION,INTERACTIONS_COUNT
3563,B00V1P4O1Y,Quaker,2.99,Enjoy a bowl of Quaker High Fiber Cinnamon Swi...,33


In [73]:
recommended_item_df['DESCRIPTION'].item()

"Enjoy a bowl of Quaker High Fiber Cinnamon Swirl Instant Oatmeal for energy and nutrition you want for the morning ahead. You can meet 40% of your daily fiber needs (based on a 2,000-calorie diet) with just one serving. A serving of High Fiber Cinnamon Swirl Oatmeal also contains calcium, iron, and five other vitamins and minerals. Quaker oats are made from 100% whole grain oats, with a Cinnamon Swirl flavor for a sweet taste. Getting a bowl of High Fiber Cinnamon Swirl Oatmeal ready is simple enough to do on even the busiest morning. Just put the oatmeal packet contents in a bowl, measure and add boiling water, and stir. Your oatmeal is ready to eat 90 seconds later. And your wholesome breakfast option isn't limited to home. Single-serving packets let you make a hearty and tasty meal anywhere there's hot water. For more than 135 years, Quaker's brands have served as symbols of quality, great taste and nutrition. Holding leadership positions in their respective categories, Quaker Oats

In [74]:
recommendations_sims_df

,ITEM_ID,BRAND,PRICE,DESCRIPTION,INTERACTIONS_COUNT
0,B00XA9DADC,Twinings,8.00,Made with 100% pure peppermint leaves. An upli...,5288
1,B00I3MVY9I,Crest,NaN,Unleash A Brilliantly Dimensional Smile A 3D W...,5086
2,B00W6T13OU,Brooklyn Beans,NaN,"In Brooklyn, we know a thing or two about bein...",4023
3,B00HZ6X8QU,Tide,11.99,Tide Original liquid laundry detergent provide...,1791
4,B0014CZUZ8,Planters,5.98,Nuts are the heroes of wholesome snacking. And...,3225
5,B00I9KR31C,Viva,NaN,View larger Product Description Dinner is over...,1467
6,B00W6T208I,Celestial Seasonings,NaN,A full-bodied black tea made from ingredients ...,1415
7,B00I8YJZ4M,Tom's of Maine,3.79,Toms of Maine Goodness Day Mt. Agamenticus loc...,1403
8,B00ICZZCEE,Pure Protein,7.09,"Pure Protein 23 Grams, Vanilla Creme(4 Count, ...",1429
9,B0014CSIG6,Mauna Loa,NaN,2g Net carbs. Taste of the tropics. Share this...,2600


成本类别和产品分组

In [75]:
recommendations_sims_v2_df

,ITEM_ID,BRAND,PRICE,DESCRIPTION,INTERACTIONS_COUNT
0,B015TYDZTE,Bounty,11.42,Ever try one of those bargain' paper towels an...,606
1,B000SSS6AU,Scotch-Brite,2.56,Keep your cookware performing at its best with...,338
2,B011DX5G92,Charmin,6.99,Charmin Ultra Strong toilet paper cleans bette...,536
3,B00NAZ5TW6,Kleenex,2.99,Hands are only as clean as the towels used to ...,73
4,B00NP79AI8,Nestle Pure Life,2.23,Nestle Pure Life Purified Water (.5-liter / 12...,678
5,B00R1H0SCA,Nature Valley,2.98,Made with 100% Natural Whole Grain Oats; 16g o...,66
6,B01BK9MXZE,Nature Valley,2.95,Each biscuit has a delicious crisp texture and...,282
7,B015O6PSO2,Frito Lay,10.67,With pre-portioned bags of your favorite snack...,394
8,B013OY25GA,Snack Pack,2.78,Give your kids a delicious treat with the Snac...,249
9,B0014CZUZ8,Planters,5.98,Nuts are the heroes of wholesome snacking. And...,3225


让我们也打印一下描述，看看它们是否与我们的项目主题匹配

In [76]:
for index, row in recommendations_sims_v2_df.iterrows():
    print('----ITEM----')
    print('Recommendation number {index}, BRAND: {brand}, PRICE: {price}'.format(index=index+1, brand=row['BRAND'], price=row['PRICE']))
    print('----DESCRIPTION----')
    print(row['DESCRIPTION'])


----ITEM----
Recommendation number 1, BRAND: Bounty, PRICE: 11.42
----DESCRIPTION----
Ever try one of those bargain' paper towels and go through so many sheets that the roll just disappears? With Bounty, the roll doesnt disappear. Bounty paper towels are 2X more absorbent,* so the roll can last up to 50% longer,* Bounty quickly absorbs messes, so you can clean up and move on without wasting time or towels. Plus, it comes in a variety of sizes and prints, including Bounty Select-A-Size, so you can choose the Bounty paper towel that's best for you and your family. Choose the paper towel you can count on: Bounty, the longer-lasting Quicker Picker Upper.* *Vs. leading ordinary brand. Dont be let down by bargain brand paper towels that tear and disappear in days. Try Bounty instead. Bounty is 2x more absorbent,* so you can pick up messes with as little as one sheet. And that means you dont have to keep pulling on a roll of Bounty, because it can last 50% longer. You'll spend more time doing

# 有针对性的示例

现在，让我们看一些更有针对性的项目，以了解这两个模型的行为方式

## 热门项目

In [77]:
asin = 'B00HZ6X8QU'
recommended_item_df = get_item_df(asin)
recommendations_sims_df = get_recs_df(asin,sims_solution['campaign_arn'])
recommendations_sims_v2_df = get_recs_df(asin,sims_v2_solution['campaign_arn'])
recommended_item_df

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,ITEM_ID,BRAND,PRICE,DESCRIPTION,INTERACTIONS_COUNT
709,B00HZ6X8QU,Tide,11.99,Tide Original liquid laundry detergent provide...,1791


理论上，对于交互次数较低的项目，SIMS 将返回最受欢迎的项目，而新的 aws-item-similarity 配方将返回与当前项目元数据（价格和描述）更相关的项目

**完整描述：**


In [78]:
recommended_item_df['DESCRIPTION'].item()

'Tide Original liquid laundry detergent provides the Tide clean you love, now more concentrated for more stain removal and freshness and less water*. Americas #1 detergent** is specifically designed to cover your many laundry needs. Also try our small but powerful Tide PODS laundry pacs. Measure your loads with cap. For medium loads, fill to bar 1. For large loads, fill to bar 3. For HE full loads, fill to bar 5. Add clothes, pour into dispenser, start washer. * vs. previous formula ** based on sales For an amazing clean you know and love, theres only one original: Tide Original Scent High Efficiency Laundry Detergent which removes over 100 stains in one wash. Its revitalizing scent is infused with aloe and floral notes to help keep your familys clothes smelling as great as they look. The formula also allows you to enjoy Tide even more as it lasts up to 2x longer compared to leading baking soda detergent based on large loads use instructions (175 oz and smaller bottles). View larger Vi

#### SIMS

In [79]:
recommendations_sims_df

,ITEM_ID,BRAND,PRICE,DESCRIPTION,INTERACTIONS_COUNT
0,B00JGZBTCI,Downy,8.47,Downy April Fresh Liquid Fabric Conditioner so...,544
1,B00MTSGYG0,Bounce,8.24,"We admit, the outdoors smell great. But with o...",394
2,B00JGZCLIY,Downy,7.94,"No dyes, no perfumes, no worries: Downy Fabric...",278
3,B00PFPDRRI,Dawn,1.92,Dawn Non-Concentrated dishwashing liquid soap ...,182
4,B00UBJQHMU,Listerine,5.99,Protect your mouth from germs and get fresh br...,483
5,B01B4N9Y6C,Cottonelle,6.99,Cottonelle Clean Care toilet paper with Clean ...,570
6,B00I9JK8GU,Ziploc,4.75,Get more out of your everyday with Ziplocbrand...,475
7,B00UB7ASK4,Old Spice,7.47,Old Spice Men's Deodorant reduces underarm odo...,49
8,B00JGZBUPE,Downy,5.11,Downy April Fresh Dryer Sheets fight static cl...,117
9,B00IA6ILKC,Glad,8.49,Glad OdorShield Trash Bags in Fresh Clean Scen...,380


我们在这里可以看到，SIMS 模型返回热门项目。这不是最优的

#### SIMS V2

In [80]:
recommendations_sims_v2_df

,ITEM_ID,BRAND,PRICE,DESCRIPTION,INTERACTIONS_COUNT
0,B00JGZBTCI,Downy,8.47,Downy April Fresh Liquid Fabric Conditioner so...,544
1,B00HZYDW5E,Coca-Cola,4.68,,408
2,B00IAE6PWA,Pepperidge Farm,2.98,"About Pepperidge Farm: For us, baking is more ...",549
3,B00099XPD6,Carnation,1.19,"Carnation Evaporated Milk is fresh, nutritious...",1291
4,B00NAZ5RIC,Kleenex,5.99,Express your style in every aspect of your hom...,513
5,B00NMPN1JC,Thera Tears,12.44,Preservative Lubricant Eye Drops.,992
6,B00UB6T2I4,Colgate,2.96,Colgate Extra Clean Full Head Medium Toothbrus...,396
7,B00UBJQHMU,Listerine,5.99,Protect your mouth from germs and get fresh br...,483
8,B000SSS6AU,Scotch-Brite,2.56,Keep your cookware performing at its best with...,338
9,B00UFUYTMK,Honey Maid,3.00,"For nearly 90 years, Honey Maid has been a who...",307


我们在两种模型中看到了非常相似的推荐！考虑到该项目是交互数据集中最受欢迎的项目之一，因此，这是一种预期行为。现在，我们来看一下冷门的项目的推荐

## 冷门项目

In [81]:
# B017BGLXYC - SODA - low price - low interactions
asin = 'B01GCT22E4'
recommended_item_df = get_item_df(asin)
recommendations_sims_df = get_recs_df(asin,sims_solution['campaign_arn'])
recommendations_sims_v2_df = get_recs_df(asin,sims_v2_solution['campaign_arn'])
recommended_item_df

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,ITEM_ID,BRAND,PRICE,DESCRIPTION,INTERACTIONS_COUNT
10456,B01GCT22E4,Pepsi,3.07,"With its light, crisp taste, Diet Pepsi gives ...",6


理论上，对于交互次数较低的项目，SIMS 将返回最受欢迎的项目，而新的 aws-item-similarity 配方将返回与当前项目元数据（价格和描述）更相关的项目

**完整描述：**


In [82]:
recommended_item_df['DESCRIPTION'].item()

'With its light, crisp taste, Diet Pepsi gives you all the refreshment you need - with zero sugar, zero calories and zero carbs. Light. Crisp. Refreshing. Diet Pepsi. Nothing refreshes like a Diet Pepsi'

#### SIMS

In [83]:
recommendations_sims_df

,ITEM_ID,BRAND,PRICE,DESCRIPTION,INTERACTIONS_COUNT
0,B00XA9DADC,Twinings,8.00,Made with 100% pure peppermint leaves. An upli...,5288
1,B00I3MVY9I,Crest,NaN,Unleash A Brilliantly Dimensional Smile A 3D W...,5086
2,B00W6T13OU,Brooklyn Beans,NaN,"In Brooklyn, we know a thing or two about bein...",4023
3,B00HZ6X8QU,Tide,11.99,Tide Original liquid laundry detergent provide...,1791
4,B0014CZUZ8,Planters,5.98,Nuts are the heroes of wholesome snacking. And...,3225
5,B00I9KR31C,Viva,NaN,View larger Product Description Dinner is over...,1467
6,B00W6T208I,Celestial Seasonings,NaN,A full-bodied black tea made from ingredients ...,1415
7,B00I8YJZ4M,Tom's of Maine,3.79,Toms of Maine Goodness Day Mt. Agamenticus loc...,1403
8,B00ICZZCEE,Pure Protein,7.09,"Pure Protein 23 Grams, Vanilla Creme(4 Count, ...",1429
9,B0014CSIG6,Mauna Loa,NaN,2g Net carbs. Taste of the tropics. Share this...,2600


我们在这里可以看到，SIMS 模型返回热门项目。这不是最优的

#### SIMS V2

In [84]:
recommendations_sims_v2_df

,ITEM_ID,BRAND,PRICE,DESCRIPTION,INTERACTIONS_COUNT
0,B015O6PSO2,Frito Lay,10.67,With pre-portioned bags of your favorite snack...,394
1,B00R1H0AYQ,Nature Valley,4.93,Made with 100% Natural Whole Grain Oats; 16g o...,322
2,B00R1H0HKI,Nature Valley,2.95,Made with 100% Natural Oats Whole grain oats a...,168
3,B00R1H0SCA,Nature Valley,2.98,Made with 100% Natural Whole Grain Oats; 16g o...,66
4,B01E3VJ8L8,Premium,2.74,Premium Original Saltine Crackers are the clas...,92
5,B0014CZUZ8,Planters,5.98,Nuts are the heroes of wholesome snacking. And...,3225
6,B013OY25GA,Snack Pack,2.78,Give your kids a delicious treat with the Snac...,249
7,B015TYDZTE,Bounty,11.42,Ever try one of those bargain' paper towels an...,606
8,B011DX5G92,Charmin,6.99,Charmin Ultra Strong toilet paper cleans bette...,536
9,B00RGVOUIO,Ritz,2.78,A Treat for All AgesFrom meats and cheeses to ...,406


让我们也打印一下描述，看看它们是否与我们的项目主题匹配

In [85]:
for index, row in recommendations_sims_v2_df.iterrows():
    print('----ITEM----')
    print('Recommendation number {index}, BRAND: {brand}, PRICE: {price}'.format(index=index+1, brand=row['BRAND'], price=row['PRICE']))
    print('----DESCRIPTION----')
    print(row['DESCRIPTION'])


----ITEM----
Recommendation number 1, BRAND: Frito Lay, PRICE: 10.67
----DESCRIPTION----
With pre-portioned bags of your favorite snacks, its no wonder why Frito-Lay variety packs are a family tradition! whether youre celebrating a big win on the soccer field, taking a family road trip, or thinking about an afternoon snack, Frito-Lay variety packs have everybodys snack needs covered. When you grab a Frito-Lay variety pack, youre ready for anything. 32 count variety pack of four 1 ounce. Bags of lays classic potato chips, five 1 ounce. Bags of CHEETOS crunchy cheese flavored snacks, three 1 ounce. Bags of FRITOS chili cheese flavored corn chips, three 1 ounce. Bags of lays barbecue flavored potato chips, four ounce. Bags of FUNYUNS snacks, five 1 ounce. Bags of DORITOS nacho cheese flavored tortilla chips, four 1 ounce. Bags of FRITOS original corn chips, four 1 ounce. Bags of lays wavy ranch flavored potato chips. 32 Count Party Mix
----ITEM----
Recommendation number 2, BRAND: Nature V

## 结论

正如我们从上面的示例中看到的，新配方 `aws-similar-items` 能够推理项目的相似性，包括它们的元数据信息。这是一个重大改进，解锁了类似项目部署的其他信号。